In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
from sys import getsizeof
from math import log
import time
from statistics import mean, stdev
from scipy.stats import norm
import pickle
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
import logging
from IPython.display import display
import os
import time
start = time.time()
import warnings

warnings.simplefilter(action='ignore', category=(FutureWarning, DeprecationWarning))


array(['kohonen', 'MAST', 'plyr', 'clusterExperiment', 'gam', 'foreach',
       'monocle', 'DDRTree', 'irlba', 'VGAM', 'splines', 'ggplot2',
       'Matrix', 'slingshot', 'princurve', 'RColorBrewer', 'scran',
       'SingleCellExperiment', 'SummarizedExperiment', 'DelayedArray',
       'matrixStats', 'Biobase', 'GenomicRanges', 'GenomeInfoDb',
       'IRanges', 'S4Vectors', 'BiocGenerics', 'parallel', 'stats4',
       'BiocParallel', 'clValid', 'cluster', 'tools', 'stats', 'graphics',
       'grDevices', 'utils', 'datasets', 'methods', 'base'], dtype='<U20')

#### To find indices of individual clusters

In [2]:
data_loc = '/Users/pk/Data/' 
hvg_bool_filename = "bool_Luecken_pp_Seurat_hvg_manual_3200_obs_2000"               
cl_alg = "kmeans" # which algorithm to sub-cluster
num_cl = 7 # which number of clusters to sub-cluster

#### Load list of lists of cluster labels

In [3]:
df_datafile = data_loc + 'Cluster_labels_df_' + hvg_bool_filename
with open(df_datafile + '.pickle', 'rb') as handle:
    df_labels = pickle.load(handle)

#### Extract cluster labels for target cluster

In [4]:
df_labels[(df_labels['cl_method'] == cl_alg) & (df_labels['num_clusters'] == num_cl)]

,cl_method,num_clusters,0,1,2,3,4,5,6,7,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
4,kmeans,7,1,7,7,1,7,4,6,1,...,1,4,4,7,7,6,4,2,1,7


#### Remove relevant row of labels from top level clustering

In [5]:
km7_cl_labels = np.array(df_labels.iloc[4, :])
km7_cl_labels = km7_cl_labels[2:]

#### partition holds indices of the observations in each cluster label

In [6]:
partitions = []
for cluster in range(1, 8):
    partitions.append(np.where(km7_cl_labels == cluster)[0])

#### combine the labels from each sub-clustering partition

In [10]:
total_cl = np.zeros(shape=(2000)) # the full set of labels for twice clustered data
for i in range(7):
    # for each partition
    source_filename = data_loc + "Cluster_labels_df_bool_Luecken_pp_Seurat_hvg_manual_3200_numEigens_13_obs_2000_partition_" + str(i+1) 
    with open(source_filename + '.pickle', 'rb') as handle:
        temp_lowlevel_df = pickle.load(handle)
    # extract labels from this partition
    temp_lowlevel_cl_labels = np.array(temp_lowlevel_df.iloc[0, 2:len(partitions[i]) + 2])
    # add a number to the labels so they're not duplicated by each partition
    for j in range(len(partitions[i])):
        total_cl[partitions[i][j]] = temp_lowlevel_cl_labels[j] + i*2 # because each partition contains two subclusters


In [12]:
louvain_filenames = ["louvain_r05_labels_Luecken", "louvain_r1_labels_Luecken"]
louvain_desc = ["Louvain Luecken r0.5", "Louvain Luecken r1"]
louvain_labels = []
for name in louvain_filenames:
    with open(data_loc + name + '.pickle', 'rb') as handle:
        temp = pickle.load(handle)
        temp = np.array(temp)
        temp = [int(x) +1 for x in temp]
        louvain_labels.append(temp)

#### Compare the labels generated by the once and twice clustered data

In [21]:
ars = adjusted_rand_score(total_cl, km7_cl_labels)
nmi = normalized_mutual_info_score(total_cl, km7_cl_labels)

print("The once- ({} clusters) and twice- ({} clusters) clustered data".format(max(km7_cl_labels), max(total_cl)))
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

print("\tAdjusted Rand ", round(ars, 3))
print("\tNormalized Mutual Information ", round(nmi, 3))


The once- (7 clusters) and twice- (44.0 clusters) clustered data
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
	Adjusted Rand  0.791
	Normalized Mutual Information  0.866


#### Comparing the Louvain cluster labels from those of the sub-clustering

In [14]:
for i in range(2):
    ars = adjusted_rand_score(total_cl, louvain_labels[i])
    nmi = normalized_mutual_info_score(total_cl, louvain_labels[i])
    
    ars2 = adjusted_rand_score(km7_cl_labels, louvain_labels[i])
    nmi2 = normalized_mutual_info_score(km7_cl_labels, louvain_labels[i])

    print("\n\n\n{} contains {} different clusters".format(louvain_desc[i], max(louvain_labels[i])))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("With respect to the sub-clustered data it has:")
    print("\tAdjusted Rand ", round(ars, 3))
    print("\tNormalized Mutual Information ", round(nmi, 3))
    print("\nWith respect to the once-clustered data it has:")
    print("\tAdjusted Rand ", round(ars2, 3))
    print("\tNormalized Mutual Information ", round(nmi2, 3))





Louvain Luecken r0.5 contains 10 different clusters
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
With respect to the sub-clustered data it has:
	Adjusted Rand  0.237
	Normalized Mutual Information  0.465

With respect to the once-clustered data it has:
	Adjusted Rand  0.195
	Normalized Mutual Information  0.403



Louvain Luecken r1 contains 16 different clusters
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
With respect to the sub-clustered data it has:
	Adjusted Rand  0.189
	Normalized Mutual Information  0.48

With respect to the once-clustered data it has:
	Adjusted Rand  0.154
	Normalized Mutual Information  0.421


#### Comparing the Louvain outputs with each other
Luecken and Seurat pre-processing, resolutions of 0.5 and 1

In [15]:
ars = adjusted_rand_score(louvain_labels[0], louvain_labels[1])
nmi = normalized_mutual_info_score(louvain_labels[0], louvain_labels[1])

print("\n\n\n{} contains {} different clusters".format(louvain_desc[0], max(louvain_labels[0])))
print("{} contains {} different clusters".format(louvain_desc[1], max(louvain_labels[1])))
print("\tAdjusted Rand ", round(ars, 3))
print("\tNormalized Mutual Information ", round(nmi, 3))





Louvain Luecken r0.5 contains 10 different clusters
Louvain Luecken r1 contains 16 different clusters
	Adjusted Rand  0.392
	Normalized Mutual Information  0.643
